In [16]:
import os
import sys
import gensim
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from gensim.models.doc2vec import LabeledSentence
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from sklearn import utils
import numpy as np
from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [2]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'\d+','<number>',text)
    return text

In [3]:
clean_train_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/train.csv")
# clean_train_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/data_train_full.csv")
clean_train_comments['content'] = clean_train_comments['content'].apply(preprocess)
clean_train_comments.head()

,content,polarity
0,gerindra alihkan rekomendasi ke agus an tanri ...,neutral
1,cuci tangan pakai sunlight stelah itu pakai sa...,neutral
2,kasus toko obat digerebek fpi propam akan peri...,neutral
3,menkeu melemah nya rupiah lebih berpengaruh pa...,neutral
4,minyak jarak castor oil <number> ml,neutral


In [4]:
tokenizer = RegexpTokenizer(r'\w+')
clean_train_comments['content'] = clean_train_comments['content'].astype('str') 
clean_train_comments.dtypes
clean_train_comments['tokens'] = clean_train_comments['content'].apply(tokenizer.tokenize)
clean_train_comments['sentiment'] = clean_train_comments['polarity'].astype('category').cat.codes
   
clean_train_comments.head()

,content,polarity,tokens,sentiment
0,gerindra alihkan rekomendasi ke agus an tanri ...,neutral,"[gerindra, alihkan, rekomendasi, ke, agus, an,...",1
1,cuci tangan pakai sunlight stelah itu pakai sa...,neutral,"[cuci, tangan, pakai, sunlight, stelah, itu, p...",1
2,kasus toko obat digerebek fpi propam akan peri...,neutral,"[kasus, toko, obat, digerebek, fpi, propam, ak...",1
3,menkeu melemah nya rupiah lebih berpengaruh pa...,neutral,"[menkeu, melemah, nya, rupiah, lebih, berpenga...",1
4,minyak jarak castor oil <number> ml,neutral,"[minyak, jarak, castor, oil, number, ml]",1


In [5]:
clean_test_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/test.csv")
# clean_test_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/data_testing_full.csv")
clean_test_comments['content'] = clean_test_comments['content'].apply(preprocess)
clean_test_comments.head()

,content,polarity
0,kemarin gue datang ke tempat makan baru yang a...,negative
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative
2,kalau dipikirpikir sebenarnya tidak ada yang b...,negative
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative


In [6]:
clean_test_comments['content'] = clean_test_comments['content'].astype('str') 
clean_test_comments.dtypes
clean_test_comments["tokens"] = clean_test_comments["content"].apply(tokenizer.tokenize)
clean_test_comments['sentiment'] = clean_test_comments['polarity'].astype('category').cat.codes

clean_test_comments.head()

,content,polarity,tokens,sentiment
0,kemarin gue datang ke tempat makan baru yang a...,negative,"[kemarin, gue, datang, ke, tempat, makan, baru...",0
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative,"[kayak, nya, sih, gue, tidak, akan, mau, balik...",0
2,kalau dipikirpikir sebenarnya tidak ada yang b...,negative,"[kalau, dipikirpikir, sebenarnya, tidak, ada, ...",0
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative,"[ini, pertama, kalinya, gua, ke, bank, buat, n...",0
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative,"[waktu, sampai, dengan, gue, pernah, disuruh, ...",0


In [7]:
x_train = clean_train_comments['content']
x_validation = clean_test_comments['content']
y_train = clean_train_comments['sentiment']
y_validation = clean_test_comments['sentiment']

In [8]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
x_train = labelize_text(x_train, 'TRAIN')
x_validation = labelize_text(x_validation, 'TEST')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [9]:
MAX_SEQUENCE_LENGTH = 85
data_dim = 500

In [10]:
word2vec = Word2Vec.load('./prosa-w2v/prosa.vec')
# word2vec = Word2Vec.load("./vectorizer/tripadvisor/word2vec_300.model")

In [11]:
def build_Word_Vector(tokens, size):
    vec = np.zeros((MAX_SEQUENCE_LENGTH - len(tokens), size))
    for word in tokens:
        try:
            vec = np.append(vec, word2vec[word])
        except KeyError: 
            vec = np.append(vec, np.zeros((1, size)))
            continue
    vec.reshape(MAX_SEQUENCE_LENGTH, size)
    return vec

In [12]:
train_vecs = np.concatenate([[build_Word_Vector(z, 500)] for z in tqdm(map(lambda x: x.words, x_train))])
val_vecs = np.concatenate([[build_Word_Vector(z, 500)] for z in tqdm(map(lambda x: x.words, x_validation))])

0it [00:00, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
935it [00:01, 642.55it/s]
100it [00:00, 812.80it/s]


In [13]:
batch_size = 256
num_epochs = 10
hidden_size = 10
timesteps = MAX_SEQUENCE_LENGTH
num_class = 1

In [14]:
num_data = len(train_vecs)
num_data_val = len(val_vecs)

train_vecs = train_vecs.reshape((num_data, timesteps, data_dim))
y_train = y_train.reshape((num_data, num_class))
val_vecs = val_vecs.reshape((num_data_val, timesteps, data_dim))
y_validation = y_validation.reshape((num_data_val, num_class))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


In [17]:
model = Sequential()
model.add(Bidirectional(LSTM(hidden_size, input_shape=(timesteps, data_dim)), merge_mode='concat'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_vecs, to_categorical(y_train), epochs=num_epochs, validation_data=(val_vecs, to_categorical(y_validation)))

Train on 935 samples, validate on 100 samples
Epoch 1/10
935/935 [==============================] - ETA: 1:27 - loss: 1.1618 - acc: 0.187 - ETA: 45s - loss: 1.0995 - acc: 0.359 - ETA: 30s - loss: 1.1172 - acc: 0.32 - ETA: 23s - loss: 1.1083 - acc: 0.34 - ETA: 18s - loss: 1.1127 - acc: 0.31 - ETA: 15s - loss: 1.1167 - acc: 0.31 - ETA: 13s - loss: 1.1110 - acc: 0.34 - ETA: 11s - loss: 1.1095 - acc: 0.35 - ETA: 10s - loss: 1.1025 - acc: 0.36 - ETA: 9s - loss: 1.0984 - acc: 0.3812 - ETA: 8s - loss: 1.0987 - acc: 0.380 - ETA: 7s - loss: 1.0962 - acc: 0.385 - ETA: 6s - loss: 1.0894 - acc: 0.394 - ETA: 6s - loss: 1.0802 - acc: 0.408 - ETA: 5s - loss: 1.0754 - acc: 0.416 - ETA: 4s - loss: 1.0714 - acc: 0.414 - ETA: 4s - loss: 1.0690 - acc: 0.408 - ETA: 3s - loss: 1.0653 - acc: 0.411 - ETA: 3s - loss: 1.0642 - acc: 0.412 - ETA: 3s - loss: 1.0645 - acc: 0.410 - ETA: 2s - loss: 1.0609 - acc: 0.418 - ETA: 2s - loss: 1.0565 - acc: 0.427 - ETA: 1s - loss: 1.0542 - acc: 0.432 - ETA: 1s - loss: 1.0501

935/935 [==============================] - ETA: 4s - loss: 0.4590 - acc: 0.781 - ETA: 4s - loss: 0.4796 - acc: 0.828 - ETA: 4s - loss: 0.4510 - acc: 0.833 - ETA: 4s - loss: 0.4218 - acc: 0.859 - ETA: 4s - loss: 0.4039 - acc: 0.868 - ETA: 3s - loss: 0.4124 - acc: 0.859 - ETA: 3s - loss: 0.4167 - acc: 0.866 - ETA: 3s - loss: 0.4205 - acc: 0.859 - ETA: 3s - loss: 0.4100 - acc: 0.861 - ETA: 3s - loss: 0.4199 - acc: 0.856 - ETA: 3s - loss: 0.4275 - acc: 0.858 - ETA: 2s - loss: 0.4214 - acc: 0.859 - ETA: 2s - loss: 0.4174 - acc: 0.858 - ETA: 2s - loss: 0.4204 - acc: 0.857 - ETA: 2s - loss: 0.4119 - acc: 0.862 - ETA: 2s - loss: 0.4068 - acc: 0.865 - ETA: 2s - loss: 0.4048 - acc: 0.864 - ETA: 1s - loss: 0.4052 - acc: 0.866 - ETA: 1s - loss: 0.4008 - acc: 0.868 - ETA: 1s - loss: 0.4064 - acc: 0.864 - ETA: 1s - loss: 0.4007 - acc: 0.867 - ETA: 1s - loss: 0.4058 - acc: 0.867 - ETA: 1s - loss: 0.4035 - acc: 0.869 - ETA: 0s - loss: 0.4116 - acc: 0.867 - ETA: 0s - loss: 0.4090 - acc: 0.868 - ETA: 0s

In [19]:
# model.save('./model/bi_lstm_3/bi_lstm_model_01.h5')  

In [18]:
# model = load_model('./model/bi_lstm_3/bi_lstm_model_01.h5')
y_pred = model.predict(val_vecs)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_validation, y_pred, labels = [0, 1, 2], digits=8))

             precision    recall  f1-score   support

          0  0.54687500 0.87500000 0.67307692        40
          1  0.50000000 0.30000000 0.37500000        20
          2  0.70833333 0.42500000 0.53125000        40

avg / total  0.60208333 0.58000000 0.55673077       100

